In [10]:
# Reload FLPyfhelin to pick up recent fixes
import importlib
import FLPyfhelin as fl
importlib.reload(fl)
from FLPyfhelin import *
print('Reloaded FLPyfhelin')

Reloaded FLPyfhelin


In [1]:
# Install medmnist if needed and export dataset to folders used by this notebook
import sys, subprocess
from pathlib import Path
import os

# Ensure medmnist is installed
try:
    import medmnist  # noqa: F401
except Exception:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "medmnist>=3.0.0"]) 
    import medmnist

from medmnist import INFO
import numpy as np
from PIL import Image

# Choose dataset
DATA_FLAG = 'pneumoniamnist'  # binary (0/1)
IMG_SIZE = 28
AS_RGB = False

# Output dirs expected by the rest of the notebook
TRAIN_DIR = Path('image/Train')
TEST_DIR  = Path('image/Test')
for p in [TRAIN_DIR, TEST_DIR]:
    p.mkdir(parents=True, exist_ok=True)

# Map label to class folder names (keep numeric names to be safe with existing code)
label_to_name = None  # will use numeric labels '0', '1', ...

# Helper to convert tensor/array to HWC numpy
def to_hwc(arr):
    x = np.array(arr)
    if x.ndim == 3 and x.shape[0] in (1,3):
        x = np.transpose(x, (1,2,0))
    return x

# Download and write images
info = INFO[DATA_FLAG]
DataClass = getattr(medmnist, info['python_class'])

train_ds = DataClass(split='train', download=True, size=IMG_SIZE, as_rgb=AS_RGB)
val_ds   = DataClass(split='val',   download=True, size=IMG_SIZE, as_rgb=AS_RGB)
test_ds  = DataClass(split='test',  download=True, size=IMG_SIZE, as_rgb=AS_RGB)

# Combine train+val into Train folder to match your generators
from itertools import chain
splits = [(chain(range(len(train_ds)), range(len(val_ds))), train_ds, val_ds, TRAIN_DIR),
          (range(len(test_ds)), test_ds, None, TEST_DIR)]

counts = {"train":0, "test":0}

def save_item(img, lab, out_root, idx):
    # label is array-like; grab first element
    try:
        y = int(lab[0])
    except Exception:
        y = int(lab)
    cls_dir = out_root / str(y)
    cls_dir.mkdir(parents=True, exist_ok=True)
    arr = to_hwc(img)
    if arr.ndim == 2:
        im = Image.fromarray(arr.astype(np.uint8), mode='L')
    else:
        im = Image.fromarray(arr.astype(np.uint8))
    im.save(cls_dir / f"{idx}.png")

# Write training (train+val)
idx = 0
for i in range(len(train_ds)):
    img, lab = train_ds[i]
    save_item(img, lab, TRAIN_DIR, idx)
    idx += 1
for i in range(len(val_ds)):
    img, lab = val_ds[i]
    save_item(img, lab, TRAIN_DIR, idx)
    idx += 1
counts["train"] = idx

# Write test
idx = 0
for i in range(len(test_ds)):
    img, lab = test_ds[i]
    save_item(img, lab, TEST_DIR, idx)
    idx += 1
counts["test"] = idx

print("Export complete:")
print(" Train images:", counts["train"]) 
print(" Test images:", counts["test"]) 
print(" Classes:", sorted({str(int(train_ds[i][1][0])) for i in range(min(100, len(train_ds))) }))

100%|██████████| 4.17M/4.17M [00:01<00:00, 3.56MB/s]
/var/folders/m7/cxy5x_m16yg7yj7qmrxswlqr0000gn/T/ipykernel_97888/1013813365.py:63: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  im = Image.fromarray(arr.astype(np.uint8), mode='L')


Export complete:
 Train images: 5232
 Test images: 624
 Classes: ['0', '1']


# Use MedMNIST as the dataset source

This section downloads a selected MedMNIST dataset and exports images to the expected folder structure:
- `image/Train/<class>/...png`
- `image/Test/<class>/...png`

Default is PNEUMONIAMNIST (binary classification). You can switch datasets in the next cell.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np              
import pandas as pd          
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, precision_score, recall_score,\
    f1_score, accuracy_score, classification_report
#import keras_tuner as kt
import os
import tensorflow as tf
from FLPyfhelin import *  # This now includes the Pyfhel shim
import time

train_path = 'image/Train'
test_path = 'image/Test'
#folder = 'image'
batch_size = 32
#num_client =2
SCALE=1
epoch=1
input_size  = (int(256*SCALE), int(256*SCALE), 3)
image_size  = (int(256*SCALE), int(256*SCALE))

In [6]:
#Generate and export public key, just need to run it once
HE=gen_pk(s=128, m=1024)
keys ={}
keys['HE'] = HE
keys['con'] = HE.to_bytes_context()
keys['pk'] = HE.to_bytes_publicKey()
keys['sk'] = HE.to_bytes_secretKey()
    
filename =  "privatekey.pickle"
with open(filename, 'wb') as handle:
    pickle.dump(keys, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
HE

In [3]:
filename =  "privatekey.pickle"
with open(filename, 'rb') as handle:
        key = pickle.load(handle)

HE = key['HE']
HE.from_bytes_context(key['con'])
HE.from_bytes_publicKey(key['pk'])
HE.from_bytes_secretKey(key['sk'])
HE

In [11]:
num_of_client_list = [2]
prec_list = []
recall_list = []
f1_list = []
acc_list = []
t =[]

for j, num_client in enumerate(num_of_client_list):
        print('Number of clients: ',num_client)
        start = time.time()

         
        print('Prepare dataset')
        df_train = prep_df(train_path,shuffle=True)
        df_test = prep_df(test_path,shuffle=False)
        test_ds = get_test_data(df_test,train_path)
        
        print('Build main model')
        model = create_model()
        model.save('main_model.hdf5')

        print("Train Clients")
        train_clients(df_train, train_path, num_client,epoch)
                
        print("Export client weights")
        export_encrypted_clients_weights(num_client)
                
        print("Aggregate Weights")
        main_model_dict = aggregate_encrypted_weights(num_client)
        filename="weights/aggregated.pickle"
        print("Export Aggregate Weights")
        export_weights(filename ,main_model_dict)
                
        agg_model = decrypt_import_weights(filename)

        print("Run predictions on aggregated model")
        preds = agg_model.predict(test_ds,verbose=1)
        predictions = preds.copy()
        predictions = [np.argmax(x) for x in predictions]
        
        print("Run prepare confusion matrix")
        prec_list.append(precision_score(test_ds.classes, predictions,average='weighted'))
        recall_list.append(recall_score(test_ds.classes, predictions,average='weighted'))
        f1_list.append(f1_score(test_ds.classes, predictions,average='weighted'))
        acc_list.append(accuracy_score(test_ds.classes, predictions))
        end = time.time()
        t.append(end-start)
        
        

Number of clients:  2
Prepare dataset
Found 624 validated image filenames belonging to 2 classes.
Build main model
Build main model


/Applications/CODES/FL with HE/.venv/lib/python3.13/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Applications/CODES/FL with HE/.venv/lib/python3.13/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Train Clients


Found 2355 validated image filenames belonging to 2 classes.
Found 261 validated image filenames belonging to 2 classes.
Found 261 validated image filenames belonging to 2 classes.


/Applications/CODES/FL with HE/.venv/lib/python3.13/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


...

/Applications/CODES/FL with HE/.venv/lib/python3.13/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 699ms/step - accuracy: 0.9284 - loss: 0.2081
Epoch 1: accuracy improved from None to 0.92654, saving model to weights/client_2.weights.h5

Epoch 1: accuracy improved from None to 0.92654, saving model to weights/client_2.weights.h5
74/74 ━━━━━━━━━━━━━━━━━━━━ 54s 731ms/step - accuracy: 0.9265 - loss: 0.1921 - val_accuracy: 0.9080 - val_loss: 0.2326 - learning_rate: 0.0010
Epoch 2/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 54s 731ms/step - accuracy: 0.9265 - loss: 0.1921 - val_accuracy: 0.9080 - val_loss: 0.2326 - learning_rate: 0.0010
Epoch 2/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 656ms/step - accuracy: 0.9349 - loss: 0.1816
Epoch 2: accuracy improved from 0.92654 to 0.93376, saving model to weights/client_2.weights.h5

Epoch 2: accuracy improved from 0.92654 to 0.93376, saving model to weights/client_2.weights.h5
74/74 ━━━━━━━━━━━━━━━━━━━━ 51s 683ms/step - accuracy: 0.9338 - loss: 0.1790 - val_accuracy: 0.9272 - val_loss: 0.2019 - learning_rate: 0.0010
Epoch 3/10


/Applications/CODES/FL with HE/.venv/lib/python3.13/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Applications/CODES/FL with HE/.venv/lib/python3.13/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Time to encrypt weights: 0.6247076988220215
Time to export weights to pickle: 0.14200711250305176
Weights exported: Client 1
Time to encrypt weights: 0.19739508628845215
Time to export weights to pickle: 0.11830019950866699
Weights exported: Client 2
Total time to encrypt and export: 1.196160078048706
Aggregate Weights
Time to encrypt weights: 0.19739508628845215
Time to export weights to pickle: 0.11830019950866699
Weights exported: Client 2
Total time to encrypt and export: 1.196160078048706
Aggregate Weights
Time to import: 0.09758400917053223
Time to import: 0.09758400917053223
Time to import: 0.07823896408081055
Time to import: 0.07823896408081055
Time to aggregate: 0.8073930740356445
Export Aggregate Weights
Time to export weights to pickle: 0.11856627464294434
Time to aggregate: 0.8073930740356445
Export Aggregate Weights
Time to export weights to pickle: 0.11856627464294434


Time to import: 0.0783848762512207
Time to decrypt: 0.14589905738830566
Run predictions on aggregated model
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 155ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 155ms/step
Run prepare confusion matrix
Run prepare confusion matrix


In [12]:
sum_list =[prec_list,recall_list,f1_list,acc_list]
client = num_of_client_list
idx=['Precision','Recall','F1 Score', 'Accuracy']

df = pd.DataFrame(sum_list, index=idx, columns=client)
df


,2
Precision,0.888952
Recall,0.889423
F1 Score,0.888636
Accuracy,0.889423


In [13]:
time_list =[t]
client = num_of_client_list
idx=['Time']

df = pd.DataFrame(time_list, index=idx, columns=client)
df

,2
Time,1037.087127


In [14]:
import pickle
model=load_weights('1')
encrypted_weights={}

for i in range(len(model.layers)):
        if model.layers[i].get_weights()!=[]:
            encrypted =[]
            weights = model.layers[i].get_weights()   

            for j in range(len(weights)):
                    array= weights[j]
                    encrypted_weights['c_'+str(i)+'_'+str(j)]=array

filename =  "plainweights.pickle"
export_weights(filename, encrypted_weights)

Time to export weights to pickle: 0.0012409687042236328


/Applications/CODES/FL with HE/.venv/lib/python3.13/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Applications/CODES/FL with HE/.venv/lib/python3.13/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(
